In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
''' 
    /*----------------------------- IMPORT_LIBRARIES -------------
'''
import os
import re
import time
import spacy
import numpy as np 
import pandas as pd

import torch
import torch.nn as nn
from torchtext.legacy import data
import torch.optim as optim
from torchtext import vocab
import torch.nn.functional as F
from torchtext.legacy.data import Field
from torch.autograd import Variable
from torchtext.vocab import Vectors
from torchtext.legacy.data import TabularDataset
device = torch.device("cuda:0")
#legacy

Mount Google Drive

In [ ]:
''' 
    /*----------------------------- MOUNT_GOOGLE_DRIVE -------------
      - To connect your colab notebook with google drive
'''    
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
''' 
    /*----------------------------- LOAD_DATASET -------------
    | Function  : load_dataset()
    | Purpose   : Reads dataset(s) in CSV file format 
    | Arguments : 
    |       drive_path : Path to dataset file
    |       dataset    : Dataset file name
    | Return    :
    |       dataset    : Dataset in dataframe format
    *---------------------------------------------------------*/
'''
def load_dataset(drive_path, dataset):
  loaded_dataset = pd.read_csv(drive_path + dataset)     # Read CSV file
  print("="*40, "\n", loaded_dataset)                    # Print the dataset that we load in previous step
  return dataset

In [ ]:
''' 
    /*----------------------------- DATA_PRE-PROCESSING -------------
    | Function  : data_pre-processing()
    | Purpose   : Performs following pre-processing:
    |              •	Remove non-alphanumeric characters
    |              •	Lower case
    |              •	Remove leading and trailing whitespaces
    | Arguments : 
    |       text: Text to be pre-processed
    | Return    :
    |       text: Pre-processed text
    *------------------------------------------------------------------------------------------------*/
'''
def data_pre_processing(text):
      text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # Remove non alphanumeric character
      text = text.lower()                        # Lowercase all text
      return text.strip()                        # Remove leading and trailing whitespaces

In [ ]:
''' 
    /*----------------------------- TOKENIZE_TEXT -------------
    | Function  : data_tokenization()
    | Purpose   : Tokenizes a Text
    | Arguments : 
    |       text: Text to be tokenized
    | Return    :
    |       text: Tokenized Text
    *------------------------------------------------------------------------------------------------*/
'''
def data_tokenization(s):
      tok = spacy.load('en')                                             # Load english tokenizer from spacy
      return [w.text.lower() for w in tok(data_pre_processing(s))]       # Apply pre-processing function (created in previous step) and tokenizer on text

In [ ]:
''' 
    /*----------------------------- BUILD_DATA_OBJECTS -------------
    | Function  : data_objects()
    | Purpose   : Build pre-processed and tokenized data objects
    | Arguments : 
    |       drive_path : Path of the directory where data files are placed
    | Return    :
    |       pre_processed_training_data, pre_processed_validation_data, pre_processed_testing_data, LABEL, TEXT
    *------------------------------------------------------------------------------------------------*/
'''
def data_objects(drive_path):
  # Declared a Field object 
  # Field : A class that stores information about the way of preprocessing
  TEXT  = Field(sequential= True, tokenize = data_tokenization, lower = True, include_lengths = True, batch_first = False, init_token = '<sos>', 
            eos_token = '<eos>')
  LABEL = data.LabelField(dtype = torch.float)
  # TabularDataset : Defines a dataset of columns. Create a TabularDataset given a path, file format, 
  # and Field list
  training_data, validation_data, testing_data = TabularDataset.splits(path = drive_path + '/Fake_News/FN-Deep-Learning/',  
                                                            train       = 'News-Training.csv', 
                                                            validation  = 'News-Validation.csv',
                                                            test        = 'News-Testing.csv',
                                                            format      = 'csv', 
                                                            fields      = [('Text', TEXT),('label', LABEL)], 
                                                            skip_header = True)
  print("\nPre-processed and Tokenized Training Data:")
  print("\n=========================================")
  for i in range(len(training_data)):
    print(training_data[i].Text)
  print("\nPre-processed and Tokenized Validation Data:")
  print("\n=========================================")
  for i in range(len(validation_data)):
    print(validation_data[i].Text)
  print("\nPre-processed and Tokenized Testing Data:")
  print("\n=========================================")
  for i in range(len(testing_data)):
      print(testing_data[i].Text)
  
  return training_data, validation_data, testing_data, LABEL, TEXT

In [ ]:
''' 
    /*----------------------------- LOAD_WORD_EMBEDDING_VECTORS -------------
    | Function  : load_word_embedding_vectors()
    | Purpose   : Load pre-trained word embedding vectors from memory
    | Arguments : 
    |       drive_path : Path to word embedding vectors file
    | Return    :
    |       vectors     : Loaded word embedding vectors
    *------------------------------------------------------------------------------------------------*/
'''
def load_word_embedding_vectors(drive_path):
  # Load word embedding vectors from memory 
  # I have downloaded the Glove word embedding vectors 100d from internet and saved in my drive 
  # To use that, I simply give the path of that file and read file in my program using vocab.Vectors function
  vectors = vocab.Vectors('glove.6B.100d.txt', drive_path)
  return vectors

In [ ]:
''' 
    /*----------------------------- BUILD_VOCABULARY -------------
    | Function  : build_vocabulary()
    | Purpose   : Build vocabulary from input data
    | Arguments : 
    |       pre_processed_training_data   : Pre-processed training data
    |     
    |       vectors                       : Word embedding vectors 
    |       LABEL                         : LABEL object (Pre-processing applied on output)
    |       TEXT                          : TEXT object (Pre-processing applied on input)
    | Return    :
    |       word_embeddings               : Word embedding vectors mapped on data
    |       vocabulary_size               : Size of vocabulary
    *------------------------------------------------------------------------------------------------*/
'''

def build_vocabulary(training_data, vectors, LABEL, TEXT):
  TEXT.build_vocab(training_data, vectors=vectors, unk_init=torch.Tensor.normal_)   # Build vocabulary from training text
  LABEL.build_vocab(training_data)                   # Build vocabulary from output / labels (Encode all labels)
  
  print("\n=========================================")
  print("Output/Label word to index dictionary: ", LABEL.vocab.stoi)
  print("\n=========================================")
  print("Input Text word to index dictionary:\n ", TEXT.vocab.stoi,"\n")
  
  word_embeddings = TEXT.vocab.vectors   # Load vectors
  vocabulary_size = len(TEXT.vocab)      # Size of vocabulary
  return word_embeddings, vocabulary_size

In [ ]:
''' 
    /*----------------------------- REPRESENT_DATA_IN_MACHINE_UNDERSTANDABLE_FORMAT -------------
    | Function  : data_iterators()
    | Purpose   : To build input data (Training, validation and testing data) iterators 
    |             (It will convert data into machine understandable format and make data objects which we can iterate over during model training and testing)
    | Arguments : 
    |       pre_processed_training_data   : Pre-processed training data
    |       pre_processed_validation_data : Pre-processed validation data
    |       pre_processed_testing_data    : Pre-processed testing data
    | Return    :
    |       training_iterator   : Training data iterator object
    |       validation_iterator : Validation data iterator object
    |       testing_iterator    : Testing data iterator object
    *------------------------------------------------------------------------------------------------*/
'''

def data_iterators(training_data, validation_data, testing_data):
  # Iterators handle numericalizing, batching, packaging. Basically, it does all the heavy lifting necessary 
  # to pass the data to a neural network
  # BucketIterator : Defines an iterator that batches examples of similar lengths together to minimizes the amount of padding needed
  # By using "splits" it applies processing steps on all datasets equally

  training_iterator, validation_iterator, testing_iterator = data.BucketIterator.splits((training_data, validation_data, testing_data), batch_size=2, sort_key=lambda x: len(x.Text), repeat=False, shuffle=True,  sort_within_batch = True)
  print("\nTraining Data Tensors Form\n")
  print("="*30, "\n")
  for batch in training_iterator:
    print(batch.Text)
  print("\nValidation Data Tensors Form\n")
  print("="*30, "\n")
  for batch in validation_iterator:
    print(batch.Text)
  print("\nTesting Data Tensors Form\n")
  print("="*30, "\n")
  for batch in testing_iterator:
    print(batch.Text)
  return training_iterator, validation_iterator, testing_iterator

In [ ]:
print("+============================Data Preparation============================+\n\n")
drive_path = '/content/drive/My Drive/Colab Notebooks/'
print("---Step 2: Load Training Data, Testing Data and Validation Data---")
print("\nTraining data before pre_processing")
original_training_data = load_dataset(drive_path, "/Fake_News/FN-Deep-Learning/News-Training.csv")

print("\n\nValidation data before pre_processing")
original_validation_data = load_dataset(drive_path, "/Fake_News/FN-Deep-Learning/News-Validation.csv")

print("\n\nTesting data before pre_processing")
original_testing_data = load_dataset(drive_path, "/Fake_News/FN-Deep-Learning/News-Testing.csv")
print("\n---Step 3: Understand and Pre-process Training Data, Testing Data and Validation Data---")
print("\n---Step 4: Represent Training Data, Testing Data and Validation Data in Machine Understandable Format---")
preprocessed_training_data, preprocessed_validation_data, preprocessed_testing_data, LABEL, TEXT = data_objects(drive_path)
# Load word embedding vectors from memory
vectors = load_word_embedding_vectors(drive_path)
# Build vocabulary
word_embeddings, vocabulary_size = build_vocabulary(preprocessed_training_data, vectors, LABEL, TEXT)
# Create iterator objects
training_iterator, validation_iterator, testing_iterator = data_iterators(preprocessed_training_data, preprocessed_validation_data, preprocessed_testing_data)

+============================Data Preparation============================+


---Step 2: Load Training Data, Testing Data and Validation Data---

Training data before pre_processing
                                                    Text label
0     Memes Breaking: Hillary cancels public appeara...  FAKE
1     Posted on October 27, 2016 by Pamela Geller \n...  FAKE
2     Russian experts collecting evidence of anti-go...  FAKE
3     A study conducted at QIMR Berghofer Medical Re...  FAKE
4     Tomorrow, the U.S. election will take place, a...  FAKE
...                                                 ...   ...
4513  The first nine months of 2013 have convinced u...  REAL
4514  Russian military might joined French warplanes...  REAL
4515  Top Dems want White House to call off Part B d...  REAL
4516  The popular conservative argument that George ...  REAL
4517  Hillary Clinton‚Äôs historic moment finally ar...  REAL

[4518 rows x 2 columns]


Validation data before pre_processing
         

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        [  237,    52],
        [ 5200,   665],
        [  640,     5],
        [ 7292,    71],
        [16518,   134],
        [17037,    30],
        [   19,   671],
        [16788,    10],
        [   61,   145],
        [    4, 10555],
        [ 3169,    28],
        [  160,   135],
        [  970,   702],
        [   15,     5],
        [   19,     4],
        [   79,   404],
        [  217,     6],
        [20989,  3995],
        [    9,     4],
        [    8,  1600],
        [ 1908,   104],
        [  987,  2711],
        [   14,    16],
        [  418,  1688],
        [ 1158,     5],
        [  742,    22],
        [ 5044,     8],
        [ 5474,    64],
        [    5,    13],
        [    4,    50],
        [ 3329,   179],
        [  260,    13],
        [   15,   101],
        [   12,    37],
        [ 2301,    24],
        [    5,  2781],
        [ 3229,    25],
        [   10,     5],
        [    4,   155],
        [  49

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        [    8,     7],
        [  395,  4814],
        [   14,  5203],
        [    4,   929],
        [ 1136,    33],
        [15625,  2733],
        [ 1035,     4],
        [   31,    72],
        [   67,    38],
        [   45,    73],
        [10924,   713],
        [    5,   184],
        [ 7312,  1673],
        [   42,  5884],
        [ 1373,  4601],
        [   46,    31],
        [13450,     4],
        [  955,   358],
        [    5,    46],
        [ 1624,  8416],
        [ 1201,   196],
        [  380,    28],
        [  576,    19],
        [ 2346,  4294],
        [   96,  5398],
        [ 2579,     5],
        [   37,    22],
        [ 1358,    64],
        [   39,     7],
        [  705,    40],
        [  535,    41],
        [ 1228,  1271],
        [    7,     9],
        [   57,    52],
        [ 3181,   358],
        [   67,     8],
        [ 2370,  2309],
        [   33,     9],
        [15832,   154],
        [  25

In [ ]:
''' 
    /*----------------------------- MODEL_ARCHITECTURE -------------
    | Class     : BLSTM()
    | Purpose   : To build the architecture of model to be trained
    *---------------------------------------------------------
    | nn.Module : Base class for all neural network modules. Your models should also subclass this class.
    |
    | Arguments:
    |      output_dim    : 1 (fake or real). For output layer number of nodes in output layer will be same as 
    |                      number of outputs required in your problem
    |	     hidden_dim    : Size of the hidden layer. Here size of hidden_state of the lstm
    | 		 input_dim     : Size of the vocabulary containing unique words. Total number of unique words in sample data 
    |		   embedding_dim : Size of each embedding vector. Here embeddding dimension of GloVe word embedding 
    |                      vectors is 100 so embedding_dim = 100
    |		   weights       : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table
    *------------------------------------------------------------------------------------
    | Function  : forward()
    | Purpose   : This function will automatically start foward propogation when model object is called
    | Arguments :
    |     text  : Input text of shape = (num_sequences, batch_size)	
	  | Return:
	  |     hidden_state : Final model state learned from input text
    ------------------------------------------------------------------------------
'''

class BLSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, word_embeddings, n_layers, n_directions):
        
        super().__init__()
        
        self.n_layers     = n_layers
        self.n_directions = n_directions
        self.hidden_dim   = hidden_dim
        self.embedding_layer = nn.Embedding(input_dim, embedding_dim)          # Embedding layer shape
        # Assign pre-trained weights and update the weights during backpropagation 
        self.embedding_layer.weight = nn.Parameter(word_embeddings, requires_grad = True)      
        self.blstm_layer       = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = True) # We can implement multiple layers of lstm simply by changing num_layers value 
        self.linear_layer      = nn.Linear(hidden_dim * 2, output_dim)               # Shape of linear layer
        
    def forward(self, text, seq_length):

        batch_size = text.shape[1]
        h_0, c_0 = self.init_hidden(batch_size)   # Initialize first hidden state to all zeros
        
        # Here we will map all the indexes present in the input sequence to the corresponding 
		    # word vector using our trained word_embedddings.
	      # embedded input of shape = (num_sequences, batch_size, embedding_dimension)
        embedded_vectors = self.embedding_layer(text)
        print(embedded_vectors)                    
        
        packed_embedded_vectors = nn.utils.rnn.pack_padded_sequence(embedded_vectors, seq_length)    # pack input sequence
        
        
        packed_output_state, (hidden_state, cell_state) = self.blstm_layer(packed_embedded_vectors, (h_0, c_0))  # Apply blstm layer and start learning sequence of words
        
        output_state, output_seq_length = nn.utils.rnn.pad_packed_sequence(packed_output_state)  # unpack sequence
        
        hidden_state = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)
        hidden_state = self.linear_layer(hidden_state)      # Apply the linear layer on hidden_state / context vector
        return torch.sigmoid(hidden_state)
    def init_hidden(self,batch_size):
        h_0 = torch.zeros(self.n_layers * self.n_directions, batch_size, self.hidden_dim)
        c_0 = torch.zeros(self.n_layers * self.n_directions, batch_size, self.hidden_dim)
        return h_0, c_0

In [ ]:
'''
/*---------------- INITIALIZE_PARAMETERS ------------------
'''
input_dimension      = len(TEXT.vocab)
embedding_dimension  = 100
hidden_dimension     = 10
output_dimension     = 1
number_of_layers     = 1
number_of_directions = 2
number_of_epochs     = 10

In [ ]:
"""
/* ----------------------- MODEL_OBJECT -----------------
| Create the object of model class and pass parameters required: BLSTM()
|           Arguments : 
|               input_dimension     : (integer) dimension of input layer(vocabulary size)
|               output_dimension    : (integer) number of output layer nodes 
|               hidden_dimension    : (integer) number of nodes/units in hidden layer
|               embedding_dimension : (integer) dimension of embedded vector
*-------------------------------------------------------*/
"""
model = BLSTM(input_dimension, embedding_dimension, hidden_dimension, output_dimension, word_embeddings, number_of_layers, number_of_directions)
model

BLSTM(
  (embedding_layer): Embedding(57102, 100)
  (blstm_layer): LSTM(100, 10, bidirectional=True)
  (linear_layer): Linear(in_features=20, out_features=1, bias=True)
)

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding_layer.weight.data[UNK_IDX] = torch.zeros(embedding_dimension)
model.embedding_layer.weight.data[PAD_IDX] = torch.zeros(embedding_dimension)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 1e-3)   # Initialize the optimizer
criterion = nn.BCEWithLogitsLoss()                     # Intialize loss function

In [ ]:
''' 
    /*----------------------------- CALCULATE_ACCURACY -------------
    | Function  : calculate_accuracy()
    | Purpose   : Calculate accuracy score
    | Arguments : 
    |       prediction : Predicted values
    |       label      : Actual values
    | Return    :
    |       accuracy   : Accuracy score
    *---------------------------------------------------------*/
'''

def calculate_accuracy(prediction, label):

    rounded_preds = torch.round(prediction)                     # Round predictions to the closest integer
    correct       = (rounded_preds == label).float()            # Convert into float for division 
    accuracy      = correct.sum() / len(correct)                # Average accuracy
    return accuracy

In [ ]:
''' 
    /*----------------------------- EPOCH_TIME_CALCULATION -------------
    | Function  : epoch_time()
    | Purpose   : Calculate time elapsed in each epoch
    | Arguments : 
    |        start_time   : Time when an epoch's execution starts
    |        end_time     : Time when an epoch's execution end
    | Return    :
    |        elapsed_mins : Time consumed by one epoch in minutes
    |        elapsed_secs : Time consumed by one epoch in seconds
    *---------------------------------------------------------*/
'''
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time                   # Time elapsed by one epoch 
    elapsed_mins = int(elapsed_time / 60)                  # Convert time in minutes
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60)) # Convert time in seconds
    return elapsed_mins, elapsed_secs

In [ ]:
''' 
    /*----------------------------- TRAIN_MODEL -------------
    | Function  : train()
    | Purpose   : Train Model
    | Arguments : 
    |        model                 : Model object
    |        training_data_iterator: Training data iterator object
    |        optimizer             : Optimization algorithm
    |        criterion             : Loss funtion
    | Return    :
    |        epoch_loss            : Train data loss at each epoch
    |        epoch_accuracy        : Train data accuracy at each epoch
    *---------------------------------------------------------*/
'''
def train(model, iterator, optimizer, criterion):
    
    epoch_loss      = 0                                                 # Initialize epoch loss to 0
    epoch_accuracy  = 0                                                 # Initialize epoch accuracy to 0
    
    model.train()                                                       # Start model training mode
    
    for batch in iterator:
        
        optimizer.zero_grad()                                           # Clear all optimized gradients
        text, seq_length = batch.Text
        predictions = model(text, seq_length).squeeze(1)                # Make model predictions on training data
        
        loss     = criterion(predictions, batch.label)                 # Calculate loss for each batch in epoch
        accuracy = calculate_accuracy(predictions, batch.label)        # Calculate accuracy for each batch in epoch
        
        loss.backward()                                                  # Start backward propogation
        optimizer.step()                                                 # Optimization of parameters
        
        epoch_loss      += loss.item()                                   # Add loss for all batches in one epoch
        epoch_accuracy  += accuracy.item()                               # Add accuracy for all batches in one epoch
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)    # Average loss and accuracy for one epoch and return

In [ ]:
''' 
    /*----------------------------- SAVE_MODEL -------------
    | Function  : save_model()
    | Purpose   : Save a trained model on your hard disk
    | Arguments : 
    |        drive_path: Path to the directory where the trained model will be saved
    | Return    :
    |        Trained model will be saved on hard disk
    *---------------------------------------------------------*/

'''
def save_model(drive_path):
  torch.save(model.state_dict(), drive_path + '/Trained Model/best-model.pt')

In [ ]:
''' 
    /*----------------------------- Evaluate_MODEL -------------
    | Function  : evaluate()
    | Purpose   : Function to be used in Validation and Test Phase
    | Arguments : 
    |        model                : Model object
    |        data_iterator:  Data iterator object
    | Return    :
    |        epoch_loss           : Data loss at each epoch
    |        epoch_accuracy       : Data accuracy at each epoch
    *---------------------------------------------------------*/
'''

def evaluate(model, iterator, criterion):
    
    epoch_loss     = 0      # Initialize epoch loss to 0
    epoch_accuracy = 0      # Initialize epoch accuracy to 0
    model.eval()            # Start model evaluation mode
    
    with torch.no_grad():
    
        for batch in iterator:


            text, seq_length = batch.Text
            predictions = model(text, seq_length).squeeze(1)                # Make model predictions on data
            loss = criterion(predictions, batch.label)               # Calculate loss for each batch in epoch
            
            accuracy = calculate_accuracy(predictions, batch.label)  # Calculate accuracy for each batch in epoch
            epoch_loss += loss.item()                                 # Add loss for all batches, in one epoch
            epoch_accuracy += accuracy.item()                         # Add accuracy for all batches in one epoch
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)  # Average loss and accuracy for one epoch and return

In [ ]:
''' 
    /*----------------------------- VALIDATE_MODEL -------------
    | Function  : validation()
    | Purpose   : Evalaute the performance of a trained  model
    | Arguments : 
    |        model                   : Model object
    |        validation_data_iterator: Validation data iterator object
    |        criterion               : Loss function
    | Return    :
    |        epoch_loss           : Validation data loss at each epoch
    |        epoch_accuracy       : Validation data accuracy at each epoch
    *---------------------------------------------------------*/
'''

def validation(model, validation_iterator, criterion):
      best_validation_loss = float('inf')                                                        # Declare best validation loss variable
      validation_loss, validation_accuracy = evaluate(model, validation_iterator, criterion)     # Start model validation phase
      
      if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        save_model(drive_path)                                   # Save model on epoch where the validation loss is lowest
      return validation_loss, validation_accuracy

In [ ]:
"""
/*---------------------- LOAD_SAVED_MODEL ----------
|  Function  : load_model()
|  Purpose   : Method to load previously saved model
|  Arguments :
|       drive_path : Path of directory where model is saved
|  Return    :
|              Saved model will be loaded in memory
*---------------------------------------------------------*/
"""
def load_model(drive_path):
  return model.load_state_dict(torch.load(drive_path + '/Trained Model/best-model.pt'))  # Load pre-trained model

In [ ]:
''' 
    /*----------------------------- TEST_MODEL -------------
    | Function  : test()
    | Purpose   : Evalaute the performance of a trained  model
    | Arguments : 
    |        model                : Model object
    |        testing_data_iterator: Test data iterator object
    |        criterion            : Loss function
    | Return    :
    |        epoch_loss           : Test data loss at each epoch
    |        epoch_accuracy       : Test data accuracy at each epoch
    *---------------------------------------------------------*/
'''

def test(model, testing_iterator, criterion):
  load_model(drive_path)
  testing_loss, testing_accuracy = evaluate(model, testing_iterator, criterion)   # Start model testing
  return testing_loss, testing_accuracy

In [ ]:
''' 
    /*----------------------------- USER_INPUT -------------
    | Function  : take_user_input()
    | Purpose   : Take unseen input from user
    | Arguments : 
    |        TEXT : Field object to apply pre-processing on input text (same as sample data)
    | Return    :
    |        user_comment_tensor : User input in machine understandable format
    |----------------------------------------------------------
    | - Let us now predict the label on a single comment for the real time evaluation purpose 
    | 1 : Take input from user
    | 2 : Preprocess the user input
    | 3 : Fit vocabulary previously made for sample data on user input. The indexes assigned for words in 
    |     sample data will be assigned to user input. Words in user input that does not appear in 
    |     sample data will have zero value
    | 4 : Convert user input to an array
    | 5 : Make tensor from array. As pytorch only work with tensors
    *---------------------------------------------------------*/

'''

def take_user_input(TEXT):
  user_comment = input("Enter comment: ") 
  
  #Preprocess user input
  preprocessed_user_comment = TEXT.preprocess(user_comment)
  preprocessed_user_comment = [TEXT.init_token] + preprocessed_user_comment + [TEXT.eos_token]
  user_comment_vocabulary = [TEXT.vocab.stoi[x] for x in preprocessed_user_comment]
  user_comment_array = np.asarray(user_comment_vocabulary)
  user_comment_tensor = torch.LongTensor(user_comment_array).unsqueeze(1)
  user_comment_tensor = user_comment_tensor
  seq_length          = [len(user_comment_tensor)]
  seq_length          = torch.LongTensor(seq_length)

  print("\nPreprocessed User_input:\n==========================")
  print(preprocessed_user_comment)
  print("\nIdx stored in vocab, corresponding to each word in user_input:\n==========================")
  print(user_comment_vocabulary) 
  print("\nUser_input as a tensor:\n==========================" )
  print(user_comment_tensor)
  
  return user_comment_tensor, seq_length

In [ ]:
"""
/*---------------------- LOAD_SAVED_MODEL ----------
|  Function  : load_model()
|  Purpose   : Method to load previously saved model
|  Arguments :
|       drive_path : Path of directory where model is saved
|  Return    :
|              Saved model will be loaded in memory
*---------------------------------------------------------*/
"""
def load_model(drive_path):
  return model.load_state_dict(torch.load(drive_path + '/Trained Model/best-model.pt'))  # Load pre-trained model

In [ ]:
"""
/*----------------------- MODEL_PREDICTION --------
|  Function  : model_prediction()
|  Purpose   : Use trained model to predict the output of unseen instances
|  Arguments : 
|       user_input : Input taken from user
|       drive_path : Path of the directory where trained model is saved
|  Return    : 
|       label     : Prediction 
*--------------------------------------------------
|   1. Set the model to evaluation mode
|   2. Set all the gradients to zero
|   3. Apply trained model on user input
|   
|   4. torch.round() : 
|         Return the value rounded to the closest integer (0 or 1)
|   5. If returned output is 1 Print "fake" else "real"
*-------------------------------------------------*/   
"""

def model_predictions(user_input, seq_length, drive_path):
  # Evaluate model
  load_model(drive_path)  # Load model from memory to test its performance
  model.eval()
  with torch.no_grad():
    #print(Evaluate_text_tensor)
    # Model Prediction
    
    out = model(user_input, seq_length)
  
  if (torch.round(out) == 1):
    label = "fake"
  else:
    label = "real"
  return label

In [ ]:
print("\n+=====================Execute the Training and Validation Phase=====================+\n\n")
# Step 5: Execute the Training Phase

for epoch in range(number_of_epochs):

    start_time = time.time()                                    # Start time when one epoch will start executing
    
    training_loss, training_accuracy     = train(model, training_iterator, optimizer, criterion)   # Start model training
    
    # Step 7: Execute the Validation Phase
    validation_loss, validation_accuracy = validation(model, validation_iterator, criterion)
    
    end_time = time.time()                                       # End time when one epoch will end executing
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)    # Calculate time consumed by one epoch (in minutes and seconds)
      
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTraining Loss: {training_loss:.3f}   | Training Accuracy: {training_accuracy*100:.2f}%')
    print(f'\tValidation Loss: {validation_loss:.3f} |  Validation Accuracy: {validation_accuracy*100:.2f}%')

print("\n+=====================Execute the Testing Phase=====================+\n\n")
# Step 8: Execute the Testing Phase
testing_loss, testing_accuracy = test(model, testing_iterator, criterion)
print(f'Testing Loss: {testing_loss:.3f} | Testing Accuracy: {testing_accuracy*100:.2f}%')


Streaming output truncated to the last 5000 lines.
           2.1362e-01,  4.8130e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-5.5293e-03,  2.0383e-02, -3.2926e-03,  ..., -2.3203e-03,
           4.8867e-03,  9.0023e-03],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]])
tensor([[[-3.5751e-03, -1.9694e-02, -1.1058e-02,  ..., -9.8441e-03,
          -3.9657e-04, -4.8433e-03],
         [-3.5751e-03, -1.9694e-02, -1.1058e-02,  ..., -9.8441e-03,
          -3.9657e-04, -4.8433e-03]],

        [[ 8.9800e-01,  1.9283e-02,  2.3970e-01,  ...,  3.3169e-01,
           3.2848e-01, -3.8979e-01],
         [-4.9933e-01, -5.4262e-01,  2.5381e-01,  ..., -8.7809e-03,
           6.3577e-01,  1.0126e+00]],

        [[ 3.6937e-01, -4.8783e-01, -9.4158e-02,  ...,  4.1768e-01,
           5.1055e-01,  7.1669e-02],
         [-3.5300e-01,  1.0842e-02, -7.2552e-01,  ...,  3.0420e-